In [2]:
# This script is used to compare two ensemble outputs (e.g., gauge-based GMET and NLDAS-based GMET)
import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from   matplotlib.dates import DateFormatter
import os
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, ens_num):
    for i in range(ens_num):
        ens_file = os.path.join(out_forc_name_base + '.' + str('%03d' % (i+1)) +'.nc')
        
        f=xr.open_dataset(ens_file)
        pcp = f['pcp'].values[:]
        t_mean = f['t_mean'].values[:]
        t_range = f['t_range'].values[:]

        if i == 0:
            lats = f['latitude'].values[:] #shape (y,x)
            lons = f['longitude'].values[:]
            time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
                 
            pcp_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))# create ens array 
            t_mean_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))
            t_range_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))

        pcp_ens[:,:,:,i] = pcp
        t_mean_ens[:,:,:,i] = t_mean
        t_range_ens[:,:,:,i] = t_range
       
    return lats, lons, time, pcp_ens, t_mean_ens, t_range_ens

# ========================================================================================================================

root_dir='/glade/u/home/hongli/scratch/2019_10_01gssha/ens_forc_wrf2'
wrf_raw_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_daily_20171202_20180407.nc')
result_dir = os.path.join(root_dir,'test_uniform_cv')
ens_num = 100

plot_date_start = '2017-12-02'
plot_date_end = '2018-04-07'
time_format = '%Y-%m-%d'
lb_perct = 5
ub_perct = 95

formatter = DateFormatter('%Y-%m-%d') #('%Y/%m')
line_marker_size = 1.5 #3
line_width = 0.5
line_alpha = 0.8
scatter_marker_size = 8
scatter_alpha = 0.8
bound_alpha = 0.8
dpi_value = 100

output_dir=os.path.join(root_dir,'scripts/step10_plot_temporal_ens')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# define plot mask for stn and nldas ensemble, repsectively.
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, '%Y-%m-%d')
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, '%Y-%m-%d')

# ========================================================================================================================
# read historical wrf data
print('Read WRF data')
f_wrf = xr.open_dataset(wrf_raw_file)
wrf_time = pd.to_datetime(f_wrf['time'].values[:]).strftime(time_format)
wrf_lats = f_wrf['lat'] #(21)
wrf_lons = f_wrf['lon'] #(18)
wrf_prcp = f_wrf['prcp'].values[:] # (time, lat, lon). unit: mm/day
wrf_tmin = f_wrf['tmin'].values[:] # (time, lat, lon). unit: degC
wrf_tmax = f_wrf['tmax'].values[:] 

wrf_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in wrf_time])
wrf_tmean = 0.5*(wrf_tmax + wrf_tmin)
wrf_trange = wrf_tmax - wrf_tmin
f_wrf.close()

wrf_mask_time = (wrf_time>=plot_date_start_obj) & (wrf_time<=plot_date_end_obj)
prcp_mean = np.nanmean(np.nanmean(wrf_prcp[wrf_mask_time,:,:], axis = 2), axis=1) # spatial mean over all grids. shape (time). 
tmean_mean = np.nanmean(np.nanmean(wrf_tmean[wrf_mask_time,:,:], axis = 2), axis=1)
trange_mean = np.nanmean(np.nanmean(wrf_trange[wrf_mask_time,:,:], axis = 2), axis=1)

# ========================================================================================================================
print('Plot')
# loop through all uniform tests
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
for test_folder in test_folders[0:1]:
    
    print(test_folder)

    # read output ensemble
    wrf_out_forc_name_base = os.path.join(result_dir,test_folder,'outputs', 'ens_forc')
    lats, lons, wrf_ens_time, wrf_pcp_ens, wrf_t_mean_ens, wrf_t_range_ens = read_ens(wrf_out_forc_name_base, ens_num)

    # define plot mask for nldas ensemble
    wrf_ens_mask_time = (wrf_ens_time>=plot_date_start_obj) & (wrf_ens_time<=plot_date_end_obj)
    
    # calculate mean and bounds
    pcp_ens  = np.nanmean(np.nanmean(wrf_pcp_ens[wrf_ens_mask_time,:,:,:], axis=2),axis=1) # spatial mean over all grids. shape (time, ens).
    pcp_ens_mean = np.nanmean(pcp_ens, axis=1) # shape (time)
    pcp_ens_lb = np.percentile(pcp_ens, lb_perct, axis = 1) # shape (time)
    pcp_ens_ub = np.percentile(pcp_ens, ub_perct, axis = 1)

    tmean_ens  = np.nanmean(np.nanmean(wrf_t_mean_ens[wrf_ens_mask_time,:,:,:], axis=2),axis=1) 
    tmean_ens_mean = np.nanmean(tmean_ens, axis=1) 
    tmean_ens_lb = np.percentile(tmean_ens, lb_perct, axis = 1)
    tmean_ens_ub = np.percentile(tmean_ens, ub_perct, axis = 1)

    trange_ens  = np.nanmean(np.nanmean(wrf_t_range_ens[wrf_ens_mask_time,:,:,:], axis=2),axis=1) 
    trange_ens_mean = np.nanmean(trange_ens, axis=1) 
    trange_ens_lb = np.percentile(trange_ens, lb_perct, axis = 1)
    trange_ens_ub = np.percentile(trange_ens, ub_perct, axis = 1)
       
    # plot
    nrow=3 # pcp, tmean, trange
    ncol=3 # time series, Q-Q plot
    fig = plt.figure(figsize=(9,9*0.85))
#     fig.set_figwidth(5*ncol) 
#     fig.set_figheight(5*0.75*nrow)
    
    mpl.rcParams['savefig.pad_inches'] = 0 # remove any padding from the edges of the figure when saved by savefig
    #     plt.annotate('Grid ('+str(y_id)+','+str(x_id)+')',(0.05,0.95),xycoords='figure fraction',fontsize='medium',fontweight='semibold')
    
    gs = fig.add_gridspec(nrow, ncol)
    ax00 = fig.add_subplot(gs[0, 0:2])
    ax01 = fig.add_subplot(gs[0, 2])
    ax10 = fig.add_subplot(gs[1, 0:2])
    ax11 = fig.add_subplot(gs[1, 2])
    ax20 = fig.add_subplot(gs[2, 0:2])
    ax21 = fig.add_subplot(gs[2, 2])
    
    # WRF vs. Ensemble [PCP]
    ax00.plot_date(wrf_time[wrf_mask_time], prcp_mean, 'b-o', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='WRF') 
    ax00.plot_date(wrf_ens_time[wrf_ens_mask_time], pcp_ens_mean, 'r-^', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='Ens Mean')
    ax00.fill_between(wrf_ens_time[wrf_ens_mask_time], pcp_ens_lb, pcp_ens_ub, linewidth=0, facecolor='grey', alpha=bound_alpha, label='Ens 90% unc bounds')
    ax01.scatter(prcp_mean, pcp_ens_mean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
    
    # WRF vs. Ensemble [TMEAN]
    ax10.plot_date(wrf_time[wrf_mask_time], tmean_mean, 'b-o', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='WRF') 
    ax10.plot_date(wrf_ens_time[wrf_ens_mask_time], tmean_ens_mean, 'r-^', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='Ens Mean')
    ax10.fill_between(wrf_ens_time[wrf_ens_mask_time], tmean_ens_lb, tmean_ens_ub, linewidth=0, facecolor='grey', alpha=bound_alpha, label='Ens 90% unc bounds')
    ax11.scatter(tmean_mean, tmean_ens_mean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
    
    # WRF vs. Ensemble [TRANGE]
    ax20.plot_date(wrf_time[wrf_mask_time], trange_mean, 'b-o', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='WRF') 
    ax20.plot_date(wrf_ens_time[wrf_ens_mask_time], trange_ens_mean, 'r-^', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='Ens Mean')
    ax20.fill_between(wrf_ens_time[wrf_ens_mask_time], trange_ens_lb, trange_ens_ub, linewidth=0, facecolor='grey', alpha=bound_alpha, label='Ens 90% unc bounds')
    ax21.scatter(trange_mean, trange_ens_mean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
        
    # 45 degree line in Q-Qplot
    axes = [ax01, ax11, ax21]
    for ax in axes:
        left, right = ax.get_xlim()
        bottom, top = ax.get_ylim()
        ax_min = min([left, bottom])
        ax_max = max([right, top])
        ax.set_xlim([ax_min, ax_max])
        ax.set_ylim([ax_min, ax_max])
        ax.plot([ax_min, ax_max],[ax_min, ax_max],color='grey',linewidth=1.0)
    
    # title
#     ax00_title_str = '(a) Daily Precipitation' 
#     ax10_title_str = '(b) Mean Temperature' 
#     ax20_title_str = '(c) Temperature Range'     
    ax00_title_str = '(a) Precip' 
    ax10_title_str = '(b) Tmean' 
    ax20_title_str = '(c) Trange'     
    axes_title_str=[ax00_title_str, ax10_title_str, ax20_title_str]
    axes=[ax00, ax10, ax20]
    for i in range(len(axes)):
        ax=axes[i]
        title_str=axes_title_str[i]
        ax.set_title(title_str, fontsize='small', fontweight='semibold')
                
    # x-axis label and legend
    axes=[ax00, ax10, ax20]
    for ax in axes:
        ax.set_xlabel('Date', fontsize='small')
        ax.set_xlim(left=plot_date_start_obj, right=plot_date_end_obj)
        ax.xaxis.set_major_formatter(formatter)
        ax.xaxis.set_tick_params(labelsize='small',rotation=30)
        ax.legend(loc='upper right', fontsize='small', framealpha=0.5) 
    
    # y-axis label and limit
    for ax in [ax00, ax10]:
        ax.set_ylabel('Precipitation (mm/day)', fontsize='small')
    for ax in [ax00]:
        ax.set_ylim(bottom=-5.0)
    for ax in [ax10, ax20]:
        ax.set_ylabel('Temperature ($^\circ$C)', fontsize='small')
    
    for ax in [ax01, ax11, ax21]:
        ax.set_xlabel('WRF', fontsize='small')
        ax.set_ylabel('Ens Mean', fontsize='small')
    
    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small') 
    
    fig.tight_layout()
    output_filename = test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
    
    del lats, lons, wrf_ens_time, wrf_pcp_ens, wrf_t_mean_ens, wrf_t_range_ens
print('Done')

Read WRF data
Plot
046grids
Done


In [4]:
test_folders

['011grids', '017grids', '026grids', '046grids', '099grids', '393grids']